## MODEL TRAINING 

#### 1.1 Import Data and Required Packages
##### Importing Pandas, Numpy, Matplotlib, Seaborn and Warings Library.

In [1]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
import warnings

### Import the data

In [2]:
df = pd.read_csv(r'C:\Projects\Student_Doc\archive (2)\modern_teen_mental_health_main.csv')
df.head()


,student_id,date,age,gender,country,grade,mood,stress_level,sleep_hours,screen_time_hours,used_ai_today,ai_tool,journaled_today,meditated_today,exercised_today,social_interaction_rating,support_feeling
0,S0001,2025-01-01,15,Female,USA,11th,5,5,3.6,2.1,False,NaN,False,False,True,6,7
1,S0001,2025-01-02,15,Female,USA,11th,6,4,9.1,8.8,True,Phind,True,False,True,4,7
2,S0001,2025-01-03,15,Female,USA,11th,8,2,8.5,7.8,True,Notion AI,False,True,False,6,5
3,S0001,2025-01-04,15,Female,USA,11th,5,6,4.9,6.1,False,NaN,True,True,False,5,9
4,S0001,2025-01-05,15,Female,USA,11th,9,1,7.7,7.5,False,NaN,True,True,False,5,9


### Dropping the Tuition ans Setting the X and y

In [4]:
X = df.drop(columns =["student_id","date","stress_level"],axis=1) # Drop the Tution as we will be prediciting the tution based on these parameters
X.head()

,age,gender,country,grade,mood,sleep_hours,screen_time_hours,used_ai_today,ai_tool,journaled_today,meditated_today,exercised_today,social_interaction_rating,support_feeling
0,15,Female,USA,11th,5,3.6,2.1,False,NaN,False,False,True,6,7
1,15,Female,USA,11th,6,9.1,8.8,True,Phind,True,False,True,4,7
2,15,Female,USA,11th,8,8.5,7.8,True,Notion AI,False,True,False,6,5
3,15,Female,USA,11th,5,4.9,6.1,False,NaN,True,True,False,5,9
4,15,Female,USA,11th,9,7.7,7.5,False,NaN,True,True,False,5,9


In [5]:
y = df["stress_level"]

### Tranform the Columns using Column Transformers 

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

#Separate the Categorical from the Numerical

num_features = X.select_dtypes(exclude=object).columns
cat_features = X.select_dtypes(include=object).columns

#Using the Onehot and Standard transformers
num_transformer = StandardScaler()
cat_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
       ('OneHotCoder', cat_transformer,cat_features),
       ("StandardScaler", num_transformer,num_features)
    ]

)

In [8]:
X = preprocessor.fit_transform(X)
#X.shape()

In [9]:
X

array([[ 1.        ,  0.        ,  0.        , ...,  1.22508514,
        -0.00685745,  0.29168861],
       [ 1.        ,  0.        ,  0.        , ...,  1.22508514,
        -1.00794518,  0.29168861],
       [ 1.        ,  0.        ,  0.        , ..., -0.8162698 ,
        -0.00685745, -0.87568844],
       ...,
       [ 0.        ,  0.        ,  1.        , ..., -0.8162698 ,
         0.99423028,  1.45906566],
       [ 0.        ,  0.        ,  1.        , ..., -0.8162698 ,
        -1.50848905,  1.45906566],
       [ 0.        ,  0.        ,  1.        , ...,  1.22508514,
        -0.00685745, -0.29199991]])

### Separate the Train and Test

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((24000, 32), (6000, 32))

#### Create an Evaluate Function to give all metrics after model Training

In [11]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true,predicted)
    mse = mean_squared_error(true,predicted)
    rmse = np.sqrt(mean_squared_error(true,predicted))
    r2_square = r2_score(true, predicted)

    return mae, rmse, r2_square           
                   

# Model Training

In [ ]:
models = {
    "LinearRegression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}

model_list = []
r2_list= []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) #Training the model

    #making prediction
    y_train_predict = model.predict(X_train)
    y_test_predict = model.predict(X_test)


    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_predict)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_predict)


    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')
    print('HI')


LinearRegression
Model performance for Training set
- Root Mean Squared Error: 0.7828
- Mean Absolute Error: 0.6452
- R2 Score: 0.8564
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.7854
- Mean Absolute Error: 0.6484
- R2 Score: 0.8582


Lasso
Model performance for Training set
- Root Mean Squared Error: 1.2708
- Mean Absolute Error: 1.0566
- R2 Score: 0.6214
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 1.2853
- Mean Absolute Error: 1.0714
- R2 Score: 0.6202


Ridge
Model performance for Training set
- Root Mean Squared Error: 0.7826
- Mean Absolute Error: 0.6451
- R2 Score: 0.8564
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.7851
- Mean Absolute Error: 0.6482
- R2 Score: 0.8583


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 0.8792
- Mean Absolute Error: 0.7139
- R2 Score: 0.8188
------------------------

## Results

In [13]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
8,AdaBoost Regressor,0.861104
2,Ridge,0.858280
0,LinearRegression,0.858180
7,CatBoosting Regressor,0.857996
5,Random Forest Regressor,0.856317
6,XGBRegressor,0.851044
3,K-Neighbors Regressor,0.734797
4,Decision Tree,0.722929
1,Lasso,0.620161


In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

# 1) Assuming you already have:
#    X_train, X_test, y_train, y_test
#    and a fitted model called `best_model`

# 2) Make predictions on your hold-out set:
y_pred = best_model.predict(X_test)

# 3) Compute key metrics:
r2  = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse**0.5

print(f"R² Score:           {r2:.3f}")
print(f"Mean Absolute Error:{mae:.2f}")
print(f"Root MSE:           {rmse:.2f}")

# 4) Plot Actual vs Predicted:
plt.figure(figsize=(6,6))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.6)
lims = [min(y_test.min(), y_pred.min()), max(y_test.max(), y_pred.max())]
plt.plot(lims, lims, 'r--')         # perfect-prediction line
plt.xlabel("Actual Target")
plt.ylabel("Predicted Target")
plt.title("Actual vs. Predicted")
plt.tight_layout()
plt.show()

# 5) (Optional) Residual plot:
residuals = y_test - y_pred
plt.figure(figsize=(6,4))
sns.histplot(residuals, kde=True)
plt.title("Residuals Distribution")
plt.xlabel("Residual (Actual – Predicted)")
plt.show()
